# Download historical equity data for NASDAQ stocks from yahoo finance

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from time import time
from tqdm import tqdm
from pathlib import Path
import pandas as pd

from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
import yfinance as yf

In [3]:
idx = pd.IndexSlice

In [4]:
results_path = Path('results', 'asset_pricing')
if not results_path.exists():
    results_path.mkdir(parents=True)

In [5]:
def chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

In [6]:
def format_time(t):
    """Return a formatted time string 'HH:MM:SS
    based on a numeric time() value"""
    m, s = divmod(t, 60)
    h, m = divmod(m, 60)
    return f'{h:0>2.0f}:{m:0>2.0f}:{s:0>2.0f}'

## Get NASDAQ symbols

In [7]:
traded_symbols = get_nasdaq_symbols()

In [8]:
traded_symbols.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11529 entries, A to ZYXI
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Nasdaq Traded     11529 non-null  bool    
 1   Security Name     11529 non-null  object  
 2   Listing Exchange  11529 non-null  category
 3   Market Category   11529 non-null  object  
 4   ETF               11529 non-null  bool    
 5   Round Lot Size    11529 non-null  float64 
 6   Test Issue        11529 non-null  bool    
 7   Financial Status  5306 non-null   category
 8   CQS Symbol        6223 non-null   object  
 9   NASDAQ Symbol     11528 non-null  object  
 10  NextShares        11529 non-null  bool    
dtypes: bool(4), category(2), float64(1), object(4)
memory usage: 608.4+ KB


## Download metadata from yahoo finance

### NASDAQ symbols

In [9]:
all_symbols = traded_symbols[~traded_symbols.ETF].index.unique().to_list()
n = len(all_symbols)
print(f'# Symbols: {n:,.0f}')

# Symbols: 8,461


In [10]:
all_symbols

['A',
 'AA',
 'AAC',
 'AAC.U',
 'AAC.W',
 'AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AACT.U',
 'AADI',
 'AAIC',
 'AAIC$B',
 'AAIC$C',
 'AAIN',
 'AAL',
 'AAM$A',
 'AAM$B',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAU',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABEV',
 'ABG',
 'ABIO',
 'ABM',
 'ABNB',
 'ABOS',
 'ABR',
 'ABR$D',
 'ABR$E',
 'ABR$F',
 'ABSI',
 'ABST',
 'ABT',
 'ABUS',
 'ABVC',
 'AC',
 'ACA',
 'ACAB',
 'ACABU',
 'ACABW',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACAH',
 'ACAHU',
 'ACAHW',
 'ACAQ',
 'ACAQ.U',
 'ACAQ.W',
 'ACAX',
 'ACAXR',
 'ACAXU',
 'ACAXW',
 'ACB',
 'ACBA',
 'ACBAU',
 'ACBAW',
 'ACCD',
 'ACCO',
 'ACDC',
 'ACDCW',
 'ACEL',
 'ACER',
 'ACET',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACGN',
 'ACHC',
 'ACHL',
 'ACHR',
 'ACHR.W',
 'ACHV',
 'ACI',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACNT',
 'ACON',
 'ACONW',
 'ACOR',
 'ACP',
 'ACP$A',
 'ACR',
 'ACR$C',
 'ACR$D',
 'ACRE',
 'ACRO',
 'ACRO.U

In [11]:
str(all_symbols[2525])

'EIX'

In [12]:
for i in range(len(all_symbols)):
    all_symbols[i] = str(all_symbols[i]).replace(',', ' ')

In [13]:
all_symbols

['A',
 'AA',
 'AAC',
 'AAC.U',
 'AAC.W',
 'AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AACT.U',
 'AADI',
 'AAIC',
 'AAIC$B',
 'AAIC$C',
 'AAIN',
 'AAL',
 'AAM$A',
 'AAM$B',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAU',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABEV',
 'ABG',
 'ABIO',
 'ABM',
 'ABNB',
 'ABOS',
 'ABR',
 'ABR$D',
 'ABR$E',
 'ABR$F',
 'ABSI',
 'ABST',
 'ABT',
 'ABUS',
 'ABVC',
 'AC',
 'ACA',
 'ACAB',
 'ACABU',
 'ACABW',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACAH',
 'ACAHU',
 'ACAHW',
 'ACAQ',
 'ACAQ.U',
 'ACAQ.W',
 'ACAX',
 'ACAXR',
 'ACAXU',
 'ACAXW',
 'ACB',
 'ACBA',
 'ACBAU',
 'ACBAW',
 'ACCD',
 'ACCO',
 'ACDC',
 'ACDCW',
 'ACEL',
 'ACER',
 'ACET',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACGN',
 'ACHC',
 'ACHL',
 'ACHR',
 'ACHR.W',
 'ACHV',
 'ACI',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACNT',
 'ACON',
 'ACONW',
 'ACOR',
 'ACP',
 'ACP$A',
 'ACR',
 'ACR$C',
 'ACR$D',
 'ACRE',
 'ACRO',
 'ACRO.U

In [14]:
yf_symbols = yf.Tickers(all_symbols)

In [ ]:
meta_data = []
start = time()
for ticker, yf_object in tqdm(yf_symbols.tickers.items()):
    try:
        s = pd.Series(yf_object.get_info())
        meta_data.append(s.to_frame(ticker))
    except Exception as e:
        # track errors
        print(ticker, e)
        
print(f'Success: {len(meta_data):5,.0f} / {n:5,.0f}')

In [18]:
df = pd.concat(meta_data, axis=1).dropna(how='all').T
df = df.apply(pd.to_numeric, errors='ignore')
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 7690 entries, A to ZYXI
Columns: 143 entries, address1 to openInterest
dtypes: float64(112), object(31)
memory usage: 8.4+ MB


In [19]:
df.to_hdf(results_path / 'data.h5', 'stocks/info')

## Download adjusted price data using yfinance

In [ ]:
prices_adj = []
start = time()
for i, chunk in enumerate(chunks(all_symbols, 100), 1):
    prices_adj.append(yf.download(chunk, period='max', auto_adjust=True).stack(-1))

    per_ticker = (time()-start) / (i * 100)
    to_do = n - (i * 100)
    to_go = to_do * per_ticker    
    print(f'Success: {len(prices_adj):5,.0f}/{i:5,.0f} | To go: {format_time(to_go)} ({to_do:5,.0f})')

[*********************100%***********************]  100 of 100 completed

21 Failed downloads:
- AACT.U: No timezone found, symbol may be delisted
- ACAQ.W: No timezone found, symbol may be delisted
- ACABW: 1d data not available for startTime=-2208994789 and endTime=1682562171. Only 100 years worth of day granularity data are allowed to be fetched per request.
- ACACW: 1d data not available for startTime=-2208994789 and endTime=1682562171. Only 100 years worth of day granularity data are allowed to be fetched per request.
- AAM$B: No timezone found, symbol may be delisted
- AAC.U: No timezone found, symbol may be delisted
- ACAQ.U: No timezone found, symbol may be delisted
- AACIW: 1d data not available for startTime=-2208994789 and endTime=1682562177. Only 100 years worth of day granularity data are allowed to be fetched per request.
- ACDCW: 1d data not available for startTime=-2208994789 and endTime=1682562177. Only 100 years worth of day granularity data are allowed to be fetched 

KeyboardInterrupt: 

[**********************99%***********************]  99 of 100 completed

In [ ]:
prices_adj = (pd.concat(prices_adj)
              .dropna(how='all', axis=1)
              .rename(columns=str.lower)
              .swaplevel())

In [ ]:
prices_adj.index.names = ['ticker', 'date']

In [ ]:
len(prices_adj.index.unique('ticker'))

### Remove outliers

In [ ]:
df = prices_adj.close.unstack('ticker')
pmax = df.pct_change().max()
pmin = df.pct_change().min()
to_drop = pmax[pmax > 1].index.union(pmin[pmin<-1].index)
len(to_drop)

In [ ]:
prices_adj = prices_adj.drop(to_drop, level='ticker')

In [ ]:
len(prices_adj.index.unique('ticker'))

In [ ]:
prices_adj.sort_index().loc[idx[:, '1990': '2019'], :].to_hdf(results_path / 'data.h5', 
                                                              'stocks/prices/adjusted')

# 조한용 파이팅